In [18]:
from huggingface_hub import login
import os
import sys
import csv
from tqdm import trange
from transformers import AutoModel,AutoTokenizer
# FILE_PATH = './QA_results_GT.csv'
# os.environ["OPENAI_API_KEY"] = AAA

import pandas as pd
import ast

In [2]:
# ANA_FILE_PATH = './mthp_output.csv'

# naiveanswer_LIST = []
# lightraganswer_LIST = []
# minianswer_LIST = []
# QUESTION_LIST = []
# GA_LIST = []
# filelength = 0
# with open(ANA_FILE_PATH, mode='r', encoding='utf-8') as question_file:
#     reader = csv.DictReader(question_file)
#     for row in reader:
#         QUESTION_LIST.append(row['Question'])
#         GA_LIST.append(row['Gold Answer'])
#         naiveanswer_LIST.append(row['naive'])
#         lightraganswer_LIST.append(row['lightrag'])
#         minianswer_LIST.append(row['minirag'])
#         filelength = filelength+1

In [3]:
PROMPT = """
Now, I'll give you a question, a gold answer to this question, and three answers provided by different students.

Determine the answer according to the following rules:
If the answer is correct, get 1 point.
If the answer is irrelevant to the question, it will receive 0 points.
If the answer is incorrect, get -1 point.

Return your answer in JSON mode.

For example:

Question:
When does Li Hua arrive to the city?

Gold Answer:
20260105

Answer1: LiHua arrived on the afternoon of January 5th
Answer2: Sorry, there is no information about LiHua's arrival in the information you provided
Answer3: There is no accurate answer in the information you provided, but according to the first information found, LiHua arrived on April 17th

output:
{{
"Score1": 1,
"Score2": 0,
"Score3": -1,
}}



Real data:

Question:
{question}
Gold Answer:
{ga}

Answer1: {naive}
Answer2: {light}
Answer3: {mini}

output:

"""

In [5]:
PROMPT="""
You are an evaluator designed to score an answer provided by a student against a gold standard.

Determine the score according to the following rules:
If the Answer is correct, assign 1 point.
If the Answer is irrelevant to the question, assign 0 points.
If the Answer is incorrect, assign -1 point.

The score must be returned in JSON format.

---
Example 1:

Question:
When does Li Hua arrive to the city?

Gold Answer:
20260105

Answer: LiHua arrived on the afternoon of January 5th

output:
{{
"Score": 1
}}

---
Example 2:

Question:
What is Li Hua's favorite day to go to the gym?

Gold Answer:
There is no information about Li Hua's favorite day to go to the gym

Answer: Sorry, there is no information about Li Hua's favorite day to go to the gym

output:
{{
"Score": 1
}}

---
Example 3:

Question:
When does Li Hua reschedule her training session to Friday?

Gold Answer:
20260211

Answer: Li Hua rescheduled her training session to Friday, April 17th.

output:
{{
"Score": -1
}}

---
Real data to evaluate:

Question:
{question}

Gold Answer:
{ga}

Answer: {student_answer}

output:
"""

In [ ]:
import requests
import json

def get_eval_deepseek(ques, gold_answer, response):

    p = PROMPT.format(question = ques, ga = gold_answer,  student_answer = response)
    response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": "Bearer <api key>",
        "HTTP-Referer": "<YOUR_SITE_URL>",
        "X-Title": "<YOUR_SITE_NAME>",
    },
    data=json.dumps({
        "model": "deepseek/deepseek-chat",
        "messages": [
        {
                    "role": "system",
                    "content":p,
        }
        ]
    })
    )
    return response.json()['choices'][0]['message']['content']

In [8]:
df=pd.read_csv("/content/relevant_context3 (1).csv")

In [11]:
a=get_eval_deepseek(df['Question'][0], df['Gold Answer'][0], df['minirag'][0])

In [13]:
df['minirag_eval']=df.apply(lambda x: get_eval_deepseek(x['Question'], x['Gold Answer'], x['minirag']), axis=1)



In [17]:
df['relevant_minirag_eval']=df.apply(lambda x: get_eval_deepseek(x['Question'], x['Gold Answer'], x['relevant_context_response']), axis=1)



In [19]:
df['relevant_minirag_eval']

,relevant_minirag_eval
0,"```json\n{\n""Score"": -1\n}\n```"
1,"```json\n{\n""Score"": 1\n}\n```"
2,"{\n""Score"": 1\n}"
3,"{\n""Score"": -1\n}"
4,"```json\n{\n""Score"": 1\n}\n```"
5,"```json\n{\n""Score"": -1\n}\n```"
6,"```json\n{\n""Score"": 0\n}\n```"
7,"```json\n{\n""Score"": -1\n}\n```"
8,"```json\n{\n""Score"": 1\n}\n```"
9,"```json\n{\n""Score"": -1\n}\n```"


In [20]:
df['relevant_minirag_acc']=df['relevant_minirag_eval'].apply(lambda x: ast.literal_eval(x.strip("```json").strip("```"))['Score'])

In [21]:
df['minirag_acc']=df['minirag_eval'].apply(lambda x: ast.literal_eval(x.strip("```json").strip("```"))['Score'])

In [22]:
df['minirag_acc'].value_counts()

,count
minirag_acc,
1,37
-1,15
0,6


In [23]:
37/58

0.6379310344827587

In [24]:
df['relevant_minirag_acc'].value_counts()

,count
relevant_minirag_acc,
1,39
-1,16
0,3


In [25]:
39/58

0.6724137931034483

In [14]:
# #deepseek
# from openai import OpenAI
# chatbot = OpenAI(api_key=My_deepseek_key, base_url="https://api.deepseek.com")

# chat_list = []
# for i in range(filelength):
#     p = PROMPT.format(question = QUESTION_LIST[i], ga = GA_LIST[i], naive = naiveanswer_LIST[i], light = lightraganswer_LIST[i], mini = minianswer_LIST[i])
#     chat_completion = chatbot.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content":p,
#             },


#         ],
#         model="deepseek-chat",
#         stream = False
#     )
#     chat_list.append(chat_completion.choices[0].message.content.strip())

In [15]:
# #openai
# from openai import OpenAI
# from tqdm import trange
# chatbot = OpenAI()
# chat_list = []
# for i in trange(filelength):
#     p = PROMPT.format(question = QUESTION_LIST[i], ga = GA_LIST[i], naive = naiveanswer_LIST[i], light = lightraganswer_LIST[i], mini = minianswer_LIST[i])
#     chat_completion = chatbot.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content":p,
#             },
#         ],
#         model="gpt-4o",
#     )
#     chat_list.append(chat_completion.choices[0].message.content.strip())
